<a href="https://colab.research.google.com/github/kenpurin/pyspark/blob/main/test_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import time

In [8]:
start_time=time.time()
!pip install pyspark
print("--- %s seconds ---" %(time.time()-start_time))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--- 4.549505710601807 seconds ---


In [9]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [10]:
start_time=time.time()
from pyspark.sql import SparkSession
# ในกรณีที่ไม่มี spark context อยู่ก่อนแล้ว เราก็สร้างใหม่
if (sc is None):
    sc = SparkContext(master="local[*]")

spark = SparkSession(sparkContext=sc)\
        .builder\
        .appName("How to Spark")\
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

print("--- %s seconds ---" %(time.time()-start_time))

--- 1.5707650184631348 seconds ---


In [11]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [12]:
sc.version

'3.3.0'

In [13]:
#get data test
!git clone https://github.com/codeforamerica/ohana-api.git

Cloning into 'ohana-api'...
remote: Enumerating objects: 14334, done.
remote: Counting objects: 100% (903/903), done.
remote: Compressing objects: 100% (431/431), done.
remote: Total 14334 (delta 488), reused 804 (delta 455), pack-reused 13431
Receiving objects: 100% (14334/14334), 6.19 MiB | 12.04 MiB/s, done.
Resolving deltas: 100% (9575/9575), done.


In [14]:
#/content/ohana-api/data/sample-csv/taxonomy.csv
mtcars_rdd = sc.textFile('/content/ohana-api/data/sample-csv/taxonomy.csv')

In [15]:
start_time=time.time()
df = spark.read.csv('/content/ohana-api/data/sample-csv/taxonomy.csv', header=True, inferSchema=True)
print("--- %s seconds ---" %(time.time()-start_time))

--- 6.870236873626709 seconds ---


In [16]:
df.dtypes

[('taxonomy_id', 'string'),
 ('name', 'string'),
 ('parent_id', 'string'),
 ('parent_name', 'string')]

In [17]:
df.show()

+-----------+--------------------+---------+----------------+
|taxonomy_id|                name|parent_id|     parent_name|
+-----------+--------------------+---------+----------------+
|        101|           Emergency|     null|            null|
|     101-01|   Disaster Response|      101|       Emergency|
|     101-02|      Emergency Cash|      101|       Emergency|
|  101-02-01|   Help Pay for Food|   101-02|  Emergency Cash|
|  101-02-02|Help Pay for Heal...|   101-02|  Emergency Cash|
|  101-02-03|Help Pay for Housing|   101-02|  Emergency Cash|
|  101-02-04|    Help Pay for Gas|   101-02|  Emergency Cash|
|  101-02-05| Help Pay for School|   101-02|  Emergency Cash|
|  101-02-06|Help Pay for Util...|   101-02|  Emergency Cash|
|     101-03|      Emergency Food|      101|       Emergency|
|     101-04|   Emergency Shelter|      101|       Emergency|
|     101-05|Help Find Missing...|      101|       Emergency|
|     101-06|    Immediate Safety|      101|       Emergency|
|  101-0

In [18]:
df.head(10)

[Row(taxonomy_id='101', name='Emergency', parent_id=None, parent_name=None),
 Row(taxonomy_id='101-01', name='Disaster Response', parent_id='101', parent_name='Emergency'),
 Row(taxonomy_id='101-02', name='Emergency Cash', parent_id='101', parent_name='Emergency'),
 Row(taxonomy_id='101-02-01', name='Help Pay for Food', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-02-02', name='Help Pay for Healthcare', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-02-03', name='Help Pay for Housing', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-02-04', name='Help Pay for Gas', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-02-05', name='Help Pay for School', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-02-06', name='Help Pay for Utilities', parent_id='101-02', parent_name='Emergency Cash'),
 Row(taxonomy_id='101-03', name='Emergency Food', parent_id='101', parent_nam

In [19]:
df.first()

Row(taxonomy_id='101', name='Emergency', parent_id=None, parent_name=None)

In [20]:
df.take(2)

[Row(taxonomy_id='101', name='Emergency', parent_id=None, parent_name=None),
 Row(taxonomy_id='101-01', name='Disaster Response', parent_id='101', parent_name='Emergency')]

In [21]:
df.printSchema()

root
 |-- taxonomy_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- parent_name: string (nullable = true)



In [22]:
df.describe().show() 

+-------+---------------+----------------+------------------+--------------------+
|summary|    taxonomy_id|            name|         parent_id|         parent_name|
+-------+---------------+----------------+------------------+--------------------+
|  count|            290|             290|               279|                 279|
|   mean|          106.0|            null|105.86666666666666|                null|
| stddev|3.3166247903554|            null| 3.306217392478572|                null|
|    min|            101|         12-Step|               101|Addiction & Recovery|
|    max|         111-05|Workplace Rights|         111-01-01|                Work|
+-------+---------------+----------------+------------------+--------------------+



In [23]:
df.describe(['taxonomy_id']).show()

+-------+---------------+
|summary|    taxonomy_id|
+-------+---------------+
|  count|            290|
|   mean|          106.0|
| stddev|3.3166247903554|
|    min|            101|
|    max|         111-05|
+-------+---------------+



In [24]:
df.columns 

['taxonomy_id', 'name', 'parent_id', 'parent_name']

In [25]:
df.count() 

290

In [26]:
spark.stop()

#FMP




In [28]:
#!/usr/bin/env python

try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import certifi
import json

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

url_hd = ("https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/AAPL?apikey=8ffff08324836e69d3593a5c14dfc86d")
print(get_jsonparsed_data(url_hd))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.


{'symbol': 'AAPL', 'historical': [{'date': '2022-08-05', 'label': 'August 05, 22', 'adjDividend': 0.23, 'dividend': 0.23, 'recordDate': '2022-08-08', 'paymentDate': '2022-08-11', 'declarationDate': '2022-07-28'}, {'date': '2022-05-06', 'label': 'May 06, 22', 'adjDividend': 0.23, 'dividend': 0.23, 'recordDate': '2022-05-09', 'paymentDate': '2022-05-12', 'declarationDate': '2022-04-28'}, {'date': '2022-02-04', 'label': 'February 04, 22', 'adjDividend': 0.22, 'dividend': 0.22, 'recordDate': '2022-02-07', 'paymentDate': '2022-02-10', 'declarationDate': '2022-01-27'}, {'date': '2021-11-05', 'label': 'November 05, 21', 'adjDividend': 0.22, 'dividend': 0.22, 'recordDate': '2021-11-08', 'paymentDate': '2021-11-11', 'declarationDate': '2021-10-28'}, {'date': '2021-08-06', 'label': 'August 06, 21', 'adjDividend': 0.22, 'dividend': 0.22, 'recordDate': '2021-08-09', 'paymentDate': '2021-08-12', 'declarationDate': '2021-07-27'}, {'date': '2021-05-07', 'label': 'May 07, 21', 'adjDividend': 0.22, 'di

In [30]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [31]:
start_time=time.time()
from pyspark.sql import SparkSession
# ในกรณีที่ไม่มี spark context อยู่ก่อนแล้ว เราก็สร้างใหม่
if (sc is None):
    sc = SparkContext(master="local[*]")

spark = SparkSession(sparkContext=sc)\
        .builder\
        .appName("How to Spark")\
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

print("--- %s seconds ---" %(time.time()-start_time))

--- 0.07771158218383789 seconds ---


In [33]:
import pandas as pd
#แปลงค่าที่รับมาจาก api ให้อยู่ในรูปแบบ dataframe
#keep value from api in data_hd
data_hd=get_jsonparsed_data(url_hd)

#ใช้ pd.json_normalize เพราะข้อมูลมีการซ้อนกันใน historical
df_hd= pd.json_normalize(data_hd, 'historical')
df_hd.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.


,date,label,adjDividend,dividend,recordDate,paymentDate,declarationDate
0,2022-08-05,"August 05, 22",0.230,0.230,2022-08-08,2022-08-11,2022-07-28
1,2022-05-06,"May 06, 22",0.230,0.230,2022-05-09,2022-05-12,2022-04-28
2,2022-02-04,"February 04, 22",0.220,0.220,2022-02-07,2022-02-10,2022-01-27
3,2021-11-05,"November 05, 21",0.220,0.220,2021-11-08,2021-11-11,2021-10-28
4,2021-08-06,"August 06, 21",0.220,0.220,2021-08-09,2021-08-12,2021-07-27
5,2021-05-07,"May 07, 21",0.220,0.220,2021-05-10,2021-05-13,2021-04-28
6,2021-02-05,"February 05, 21",0.205,0.205,2021-02-08,2021-02-11,2021-01-27
7,2020-11-06,"November 06, 20",0.205,0.205,2020-11-09,2020-11-12,2020-10-29
8,2020-08-07,"August 07, 20",0.205,0.820,2020-08-10,2020-08-13,2020-07-30
9,2020-05-08,"May 08, 20",0.205,0.820,2020-05-11,2020-05-14,2020-04-30


In [36]:
#คำสั่งใน spark
#https://sparkbyexamples.com/spark/different-ways-to-create-a-spark-dataframe/

df_spark  = spark.createDataFrame(df_hd)

In [38]:
df_spark.show()

+----------+---------------+-----------+--------+----------+-----------+---------------+
|      date|          label|adjDividend|dividend|recordDate|paymentDate|declarationDate|
+----------+---------------+-----------+--------+----------+-----------+---------------+
|2022-08-05|  August 05, 22|       0.23|    0.23|2022-08-08| 2022-08-11|     2022-07-28|
|2022-05-06|     May 06, 22|       0.23|    0.23|2022-05-09| 2022-05-12|     2022-04-28|
|2022-02-04|February 04, 22|       0.22|    0.22|2022-02-07| 2022-02-10|     2022-01-27|
|2021-11-05|November 05, 21|       0.22|    0.22|2021-11-08| 2021-11-11|     2021-10-28|
|2021-08-06|  August 06, 21|       0.22|    0.22|2021-08-09| 2021-08-12|     2021-07-27|
|2021-05-07|     May 07, 21|       0.22|    0.22|2021-05-10| 2021-05-13|     2021-04-28|
|2021-02-05|February 05, 21|      0.205|   0.205|2021-02-08| 2021-02-11|     2021-01-27|
|2020-11-06|November 06, 20|      0.205|   0.205|2020-11-09| 2020-11-12|     2020-10-29|
|2020-08-07|  August 

In [44]:
df_spark.printSchema()

root
 |-- date: string (nullable = true)
 |-- label: string (nullable = true)
 |-- adjDividend: double (nullable = true)
 |-- dividend: double (nullable = true)
 |-- recordDate: string (nullable = true)
 |-- paymentDate: string (nullable = true)
 |-- declarationDate: string (nullable = true)



In [53]:
df_spark.count()

92

In [57]:
#อ้างมาจาก วิดีโอ bigdata อาจารย์ของมจธ
df_spark.limit(3).toPandas()

,date,label,adjDividend,dividend,recordDate,paymentDate,declarationDate
0,2022-08-05,"August 05, 22",0.23,0.23,2022-08-08,2022-08-11,2022-07-28
1,2022-05-06,"May 06, 22",0.23,0.23,2022-05-09,2022-05-12,2022-04-28
2,2022-02-04,"February 04, 22",0.22,0.22,2022-02-07,2022-02-10,2022-01-27


basic spark

In [46]:

columns = ["language","users_count"]
data = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]


In [48]:

#spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
rdd = spark.sparkContext.parallelize(data)


In [49]:

dfFromRDD1 = rdd.toDF()
dfFromRDD1.printSchema()


root
 |-- _1: string (nullable = true)
 |-- _2: string (nullable = true)



In [52]:
#https://signalfix.net/th/pyspark-collect-%E0%B8%94%E0%B8%B6%E0%B8%87%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%B9%E0%B8%A5%E0%B8%88%E0%B8%B2%E0%B8%81-dataframe
#ระมัดระวังเรื่อง collect เพราะถ้าเกิดดึงกลับมาแล้วใหญ่เกินหน่วยความจำจะเป็นปัญหา
rdd.collect()

[('Java', '20000'), ('Python', '100000'), ('Scala', '3000')]